# Analysis of the gradient conflict of Llama model and its K-neuron variants

 - gradient conflict
 - dominating gradient

目前发现使用topk neuron可能会让模型的gradient conflict增加（无论如何调整gradeint 的norm以及cos相似度大小阈值）

**Further analysis**
 - [ ] Analyze layer by layer, starting from the last layer first
 - [ ] Analyze neuron by neuron, starting from the neuron that receives max activation values (这个和考虑梯度norm的应该是很类似的，因为接收到更大激活值的neuron，按理来说应该有更大的梯度)
 - [ ] Analyze weight by weight, 目前是以neuron为单位进行分析的，去看一下 gradient surgery 那篇文章是以什么为单位进行讨论的
 - [ ] Analyze batch by batch, 目前是以sample为单位进行分析的，不过感觉batch_size=1也是一种情况，不应该被特殊处理，即batch_size为多少都应该
 - [ ] **第一优先级** 有一种可能是topk的选取方式让neuron在不同sample上获取了更加平均的梯度，即梯度不会被一个sample所dominate。因此即使梯度有冲突也可以很好的average之后进行很好的优化。

In [1]:

import os
import torch
import random 
import torch.nn as nn
from transformers import LlamaForCausalLM, AutoTokenizer
from topk_models import TopKLlamaForCausalLM, TopKLlamaConfig
from transformers.models.llama.modeling_llama import LlamaForCausalLM
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np

# Configurations of this notobook
SEED=1
MODEL_PATH = 'models/Llama-2-7b-hf'
MODEL_TYPE = 'original'
DATA_PATH = './data/alpaca/'
N_SAMPLES = 50
CACHE_PATH = 'cache/original_llama/'
USE_GATE, USE_UP, USE_DOWN = True, True, True
if MODEL_TYPE == 'topk':
    from datetime import datetime
    currentTime = datetime.now().strftime("%H-%M-%S")
    CACHE_PATH = f'cache/{currentTime}'
    
    os.system(f"mkdir {CACHE_PATH} & cp topk_7b_configs.json {CACHE_PATH}")
    
# load the model
if MODEL_TYPE == 'original':
    model = LlamaForCausalLM.from_pretrained("./models/Llama-2-7b-hf/", device_map="auto")
else:
    # load the topk model
    config = TopKLlamaConfig.from_json_file('topk_7b_configs.json')
    model = TopKLlamaForCausalLM.from_pretrained(
        "./models/Llama-2-7b-hf/", 
        # device_map="auto",
        config=config
    )
tokenizer = AutoTokenizer.from_pretrained("./models/Llama-2-7b-hf/")
model.eval()

/home/hkustadmin/miniconda3/envs/emo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.07s/it]
/home/hkustadmin/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/hkustadmin/miniconda3/envs/emo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. How

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

Sample some data points from the instruction-tuning data, e.g., Alpaca52K

In [2]:
import random
from datasets import load_dataset
random.seed(10)
data = load_dataset(DATA_PATH)['train']
data_idx_sample = random.sample([_ for _ in range(len(data))], N_SAMPLES)
data_samples = [data[i] for i in data_idx_sample]

Register gradients of the model

In [3]:
IGNORE_INDEX = -100
from copy import deepcopy
from tqdm import tqdm
from utils import PROMPT_TEMPLATE_SINGLE as PROMPT_DICT
from torch import optim
prompt = PROMPT_DICT["prompt_input"]


n_layer = 32
grads = {
    "gate_proj": [[] for _ in range(n_layer)], "up_proj": [[] for _ in range(n_layer)], "down_proj": [[] for _ in range(n_layer)]
}
optimizer = optim.AdamW(model.parameters())
for d in tqdm(data_samples):
    inp = prompt.format_map(d)
    out = f"{d['output']}{tokenizer.eos_token}"
    text = inp + out
    # tokenize
    input_ids = tokenizer(text, return_tensors='pt').input_ids# .to(device)
    src_len = tokenizer(inp, return_tensors='pt').input_ids.ne(tokenizer.pad_token_id).sum().item()
    labels = deepcopy(input_ids)
    labels[:, : src_len] = IGNORE_INDEX
    labels = labels[:, 1:]
    
    # forward
    y = model(input_ids[:,:-1])
    loss = F.cross_entropy(input=y.logits.squeeze(), target=labels.squeeze())
    
    # backward
    loss.backward()
    
    # record gradients
    for n, p in model.named_parameters():
        if len(n.split(".")) < 5:
            continue
        layer_id = int(n.split(".")[2])
        name = n.split(".")[4]
        if name in grads:
            grads[name][layer_id].append(p.grad.data.detach().cpu())
            
    optimizer.zero_grad()


100%|██████████| 50/50 [07:12<00:00,  8.66s/it]


Collect the gradient and analyze their gradient conflicting, for now let's just focus on one neuron in the FFN layer.

The representation of neuron $i$ in Llama FFN layer could be defined as $[g_i, up_i, down_i]$, where $g_i$, $up_i$, $down_i$ represent the 

In [4]:
n_layers = model.config.num_hidden_layers
# for the ffn of each layer
# (neuron_num, sample, d_dim) @ (neuron_num, d_dim, sample)
norm_dominate = []
conflicts = []
epsilon = 0
gamma = 0
device='cpu'
from tqdm import trange
for layer_id in trange(n_layers):
    gate_grads = torch.stack(grads["gate_proj"][layer_id]).transpose(0, 1)
    up_grads = torch.stack(grads["up_proj"][layer_id]).transpose(0, 1)
    down_grads = torch.stack(grads["down_proj"][layer_id]).permute(2, 0, 1)
    neuron_grads = torch.cat([
        gate_grads if USE_GATE else torch.zeros_like(gate_grads), 
        up_grads if USE_UP else torch.zeros_like(up_grads), 
        down_grads if USE_DOWN else torch.zeros_like(down_grads)], dim=-1).to(device)
    
    grad_norm = torch.norm(neuron_grads, p=2, dim=-1, keepdim=True)
    grad_norm_ = torch.cat([grad_norm for _ in range(N_SAMPLES)], dim=-1)
    grad_norm_2 = grad_norm_ * grad_norm_
    tmp = torch.bmm(grad_norm, grad_norm.transpose(1, 2))
    gm_sim = 2 * torch.bmm(grad_norm, grad_norm.transpose(1, 2)) / (grad_norm_2 + grad_norm_2.transpose(1,2))
    norm_dominate.append(torch.mean(gm_sim).cpu().data)
    
    norm_mask = grad_norm > epsilon
    # print(torch.sum(norm_mask) / N_SAMPLES / 11008)
    norm_mask = norm_mask.float()
    batch_norm_mask = torch.bmm(norm_mask, norm_mask.transpose(1, 2)).cpu()
    
    _a = F.normalize(neuron_grads, p=2, dim=-1)
    _b = _a.transpose(1, 2)
    conflict = torch.bmm(_a, _b).cpu()
    conflict.masked_fill_(~batch_norm_mask.bool(), 0)
    conflict_rate = torch.sum(conflict < gamma) / torch.sum(batch_norm_mask)
    conflicts.append(conflict_rate.data)
    print(torch.mean(gm_sim))
    print(conflicts[-1])
    
# plt.bar([ _ for _ in range(2)], conflicts)

  3%|▎         | 1/32 [00:06<03:12,  6.21s/it]

tensor([[[1.0000, 0.9938, 0.9327,  ..., 0.9882, 1.0000, 0.9925],
         [0.9938, 1.0000, 0.9657,  ..., 0.9991, 0.9939, 0.9731],
         [0.9327, 0.9657, 1.0000,  ..., 0.9755, 0.9332, 0.8866],
         ...,
         [0.9882, 0.9991, 0.9755,  ..., 1.0000, 0.9884, 0.9628],
         [1.0000, 0.9939, 0.9332,  ..., 0.9884, 1.0000, 0.9923],
         [0.9925, 0.9731, 0.8866,  ..., 0.9628, 0.9923, 1.0000]],

        [[1.0000, 0.9030, 0.9982,  ..., 0.7508, 0.9946, 0.9643],
         [0.9030, 1.0000, 0.9253,  ..., 0.9466, 0.9401, 0.9826],
         [0.9982, 0.9253, 1.0000,  ..., 0.7804, 0.9990, 0.9781],
         ...,
         [0.7508, 0.9466, 0.7804,  ..., 1.0000, 0.8017, 0.8775],
         [0.9946, 0.9401, 0.9990,  ..., 0.8017, 1.0000, 0.9862],
         [0.9643, 0.9826, 0.9781,  ..., 0.8775, 0.9862, 1.0000]],

        [[1.0000, 0.9869, 0.9624,  ..., 0.9969, 0.9963, 0.9967],
         [0.9869, 1.0000, 0.9933,  ..., 0.9965, 0.9971, 0.9967],
         [0.9624, 0.9933, 1.0000,  ..., 0.9804, 0.9819, 0.

  6%|▋         | 2/32 [00:14<03:49,  7.66s/it]

tensor([[[1.0000, 0.9873, 0.9061,  ..., 0.7618, 0.9665, 0.9755],
         [0.9873, 1.0000, 0.9591,  ..., 0.8384, 0.9169, 0.9306],
         [0.9061, 0.9591, 1.0000,  ..., 0.9508, 0.7900, 0.8087],
         ...,
         [0.7618, 0.8384, 0.9508,  ..., 1.0000, 0.6313, 0.6505],
         [0.9665, 0.9169, 0.7900,  ..., 0.6313, 1.0000, 0.9992],
         [0.9755, 0.9306, 0.8087,  ..., 0.6505, 0.9992, 1.0000]],

        [[1.0000, 0.9711, 0.9999,  ..., 0.9994, 0.9883, 0.9744],
         [0.9711, 1.0000, 0.9736,  ..., 0.9622, 0.9260, 0.9999],
         [0.9999, 0.9736, 1.0000,  ..., 0.9989, 0.9866, 0.9768],
         ...,
         [0.9994, 0.9622, 0.9989,  ..., 1.0000, 0.9931, 0.9660],
         [0.9883, 0.9260, 0.9866,  ..., 0.9931, 1.0000, 0.9311],
         [0.9744, 0.9999, 0.9768,  ..., 0.9660, 0.9311, 1.0000]],

        [[1.0000, 0.9588, 0.9084,  ..., 0.9983, 0.9663, 0.9990],
         [0.9588, 1.0000, 0.9884,  ..., 0.9735, 0.9996, 0.9702],
         [0.9084, 0.9884, 1.0000,  ..., 0.9298, 0.9836, 0.

  9%|▉         | 3/32 [00:23<03:56,  8.17s/it]

...

tensor([[[1.0000, 0.9943, 0.9986,  ..., 0.8449, 0.9644, 0.9908],
         [0.9943, 1.0000, 0.9986,  ..., 0.7950, 0.9328, 0.9712],
         [0.9986, 0.9986, 1.0000,  ..., 0.8205, 0.9498, 0.9823],
         ...,
         [0.8449, 0.7950, 0.8205,  ..., 1.0000, 0.9491, 0.9025],
         [0.9644, 0.9328, 0.9498,  ..., 0.9491, 1.0000, 0.9910],
         [0.9908, 0.9712, 0.9823,  ..., 0.9025, 0.9910, 1.0000]],

        [[1.0000, 0.9325, 0.9112,  ..., 0.8774, 0.5756, 0.9870],
         [0.9325, 1.0000, 0.9982,  ..., 0.9897, 0.7617, 0.9771],
         [0.9112, 0.9982, 1.0000,  ..., 0.9964, 0.7909, 0.9632],
         ...,
         [0.8774, 0.9897, 0.9964,  ..., 1.0000, 0.8310, 0.9385],
         [0.5756, 0.7617, 0.7909,  ..., 0.8310, 1.0000, 0.6542],
         [0.9870, 0.9771, 0.9632,  ..., 0.9385, 0.6542, 1.0000]],

        [[1.0000, 0.8877, 0.7627,  ..., 0.7209, 0.9219, 0.9127],
         [0.8877, 1.0000, 0.9642,  ..., 0.9399, 0.9960, 0.9980],
         [0.7627, 0.9642, 1.0000,  ..., 0.9965, 0.9382, 0.

 97%|█████████▋| 31/32 [05:19<00:10, 10.83s/it]

tensor([[[1.0000, 0.8635, 0.7988,  ..., 0.9767, 0.9758, 0.7713],
         [0.8635, 1.0000, 0.9902,  ..., 0.9458, 0.9471, 0.9809],
         [0.7988, 0.9902, 1.0000,  ..., 0.8960, 0.8976, 0.9984],
         ...,
         [0.9767, 0.9458, 0.8960,  ..., 1.0000, 1.0000, 0.8726],
         [0.9758, 0.9471, 0.8976,  ..., 1.0000, 1.0000, 0.8744],
         [0.7713, 0.9809, 0.9984,  ..., 0.8726, 0.8744, 1.0000]],

        [[1.0000, 0.9839, 0.9164,  ..., 0.9929, 0.8828, 0.4403],
         [0.9839, 1.0000, 0.9711,  ..., 0.9981, 0.9482, 0.3733],
         [0.9164, 0.9711, 1.0000,  ..., 0.9553, 0.9963, 0.2968],
         ...,
         [0.9929, 0.9981, 0.9553,  ..., 1.0000, 0.9283, 0.3951],
         [0.8828, 0.9482, 0.9963,  ..., 0.9283, 1.0000, 0.2734],
         [0.4403, 0.3733, 0.2968,  ..., 0.3951, 0.2734, 1.0000]],

        [[1.0000, 0.9947, 0.9873,  ..., 0.8062, 0.9226, 0.9998],
         [0.9947, 1.0000, 0.9984,  ..., 0.8537, 0.9555, 0.9967],
         [0.9873, 0.9984, 1.0000,  ..., 0.8785, 0.9703, 0.

100%|██████████| 32/32 [05:30<00:00, 10.33s/it]

tensor([[[1.0000, 0.9246, 0.7988,  ..., 0.9940, 0.9737, 0.9313],
         [0.9246, 1.0000, 0.9581,  ..., 0.8823, 0.9859, 0.9998],
         [0.7988, 0.9581, 1.0000,  ..., 0.7450, 0.9014, 0.9527],
         ...,
         [0.9940, 0.8823, 0.7450,  ..., 1.0000, 0.9443, 0.8903],
         [0.9737, 0.9859, 0.9014,  ..., 0.9443, 1.0000, 0.9889],
         [0.9313, 0.9998, 0.9527,  ..., 0.8903, 0.9889, 1.0000]],

        [[1.0000, 0.8732, 0.9466,  ..., 0.7000, 0.9838, 0.8167],
         [0.8732, 1.0000, 0.9807,  ..., 0.9376, 0.9412, 0.9922],
         [0.9466, 0.9807, 1.0000,  ..., 0.8609, 0.9883, 0.9499],
         ...,
         [0.7000, 0.9376, 0.8609,  ..., 1.0000, 0.7897, 0.9724],
         [0.9838, 0.9412, 0.9883,  ..., 0.7897, 1.0000, 0.8961],
         [0.8167, 0.9922, 0.9499,  ..., 0.9724, 0.8961, 1.0000]],

        [[1.0000, 0.9640, 0.9381,  ..., 0.9906, 0.9922, 1.0000],
         [0.9640, 1.0000, 0.8279,  ..., 0.9213, 0.9892, 0.9653],
         [0.9381, 0.8279, 1.0000,  ..., 0.9755, 0.8923, 0.

In [5]:
import os
import numpy as np
os.system(f"mkdir {CACHE_PATH}")
np.save(f'{CACHE_PATH}/conflict_{SEED}_{epsilon}_{gamma}.npy', conflicts)
np.save(f'{CACHE_PATH}/gm_sim_{SEED}_{epsilon}_{gamma}.npy', norm_dominate)

mkdir: cannot create directory ‘cache/original_llama/’: File exists


In [6]:
import numpy as np
res = np.load(f'{CACHE_PATH}/conflict_{epsilon}.npy')
res2 = np.load(f'cache/19-46-26/conflict_0.01.npy')
x = [ _ for _ in range(n_layers)]
plt.plot(x, res)
plt.plot(x, res2)

FileNotFoundError: [Errno 2] No such file or directory: 'cache/original_llama//conflict_0.npy'